## 18세 미만 웹툰 크롤링
- 모든 회차를 크롤링 하려면 너무 오래걸려서 요일별로 따로 크롤링...

In [9]:
import requests
from bs4 import BeautifulSoup
import re 
from selenium import webdriver
import time

def get_webtoon_link(dayofweek):
    weekday = ['mon', 'tue', 'wed', 'thu', 'fri', 'sat', 'sun']

    datas = []
    for week in weekday:
        url = f'https://comic.naver.com/webtoon/weekdayList.nhn?week={week}&view=list&order=ViewCount'
        response = requests.get(url)
        dom = BeautifulSoup(response.content, "html.parser")

        # link1 : 각 웹툰의 링크
        links1 = dom.select('#content > div.list_area.table_list_area > table > tbody > tr > td.subject > a')

        for link in links1:
            datas.append({'link1': 'https://comic.naver.com' + link.get('href'),
                         'title': link.text,
                         'weekday': week,
                         'titleId': re.findall('\d+', link.get('href'))[0]})

    df = pd.DataFrame(datas)

    # 일주일에 두번 이상 연재되는 작품 1개만 남겨두기
    ls2 = list(df.groupby('title').count()[df.groupby('title').count().link1 >= 2].index)
    du = [df[df.title == ls].index[0] for ls in ls2]
    df.drop(du, axis=0, inplace=True)
    df.reset_index(drop=True, inplace=True)

    link1 = list (df.link1)

    # 마지막 회차 몇회인지
    info = []
    for link in link1:
        response = requests.get(link)
        dom = BeautifulSoup(response.content, "html.parser")
        last = dom.select('.title > a')[0].get('href')
        info.append({'titleId': re.findall('\d+', last)[0],
                 'last': re.findall('\d+', last)[1],
                 'content': dom.select('div.detail > p:nth-child(2)')[0].text,
                 'genre': dom.select('div.detail > p.detail_info > span.genre')[0].text,
                 'age': dom.select('div.detail > p.detail_info > span.age')[0].text[:2]
                })

    df = pd.merge(df, pd.DataFrame(info))
    df.age = df.age.apply(lambda x: 0 if x == '전체' else int(x))
    
    df1 = df[(df.age < 18)&(df.weekday == dayofweek)]
    
    last_ls = []
    for i in range(len(df1)):
        last_ls.append(list(df1[['titleId', 'last', 'weekday']].iloc[i]))

    # link2 : 각 웹툰의 모든 회차 링크
    link2 = []
    for last in last_ls:
        for no in range(1, int(last[1])+1):
            link = f'https://comic.naver.com/webtoon/detail.nhn?titleId={last[0]}&no={no}&weekday={last[2]}'
            link2.append(link)
    
    return link2
      
def get_naver_comment(links):
    # 아이프레임 가져오기
    from selenium import webdriver

    comments = []
    i = 1
    driver = webdriver.Chrome()
    for link in links:
        driver.get(link)
        time.sleep(1)
        driver.switch_to_frame('commentIframe')
        page_source = driver.page_source
        
        dom = BeautifulSoup(page_source, 'html.parser')
        coms = dom.select('.u_cbox_contents')
        
        reply = []
        for com in coms:
            reply.append(com.text)
        titleId = re.findall('\d+', link)[0]
        no = re.findall('\d+', link)[1]
        comments.append({'titleId': titleId,
                            'no': no,
                            'comments': ', '.join(reply)})
        print(i, titleId, no, sep='\t')
        i += 1
    return comments

In [5]:
links1 = get_webtoon_link('mon')
links2 = get_webtoon_link('tue')
links3 = get_webtoon_link('wed')
links4 = get_webtoon_link('thu')
links5 = get_webtoon_link('fri')
links6 = get_webtoon_link('sat')
links7 = get_webtoon_link('sun')

TypeError: len() takes exactly one argument (9 given)

In [6]:
len(links1), len(links2), len(links3), len(links4), len(links5), len(links6), len(links7)

(4025, 4603, 3345, 3727, 4061, 4045, 4403)

In [7]:
print('total :', len(links1 + links2 + links3 + links4 + links5 + links6 + links7))

total : 28209


In [10]:
start = time.time()
comments6 = get_naver_comment(links6)
print(f'time : {time.time() - start}')

1	650305	1
2	650305	2
3	650305	3
4	650305	4
5	650305	5
6	650305	6
7	650305	7
8	650305	8
9	650305	9
10	650305	10
11	650305	11
12	650305	12
13	650305	13
14	650305	14
15	650305	15
16	650305	16
17	650305	17
18	650305	18
19	650305	19
20	650305	20
21	650305	21
22	650305	22
23	650305	23
24	650305	24
25	650305	25
26	650305	26
27	650305	27
28	650305	28
29	650305	29
30	650305	30
31	650305	31
32	650305	32
33	650305	33
34	650305	34
35	650305	35
36	650305	36
37	650305	37
38	650305	38
39	650305	39
40	650305	40
41	650305	41
42	650305	42
43	650305	43
44	650305	44
45	650305	45
46	650305	46
47	650305	47
48	650305	48
49	650305	49
50	650305	50
51	650305	51
52	650305	52
53	650305	53
54	650305	54
55	650305	55
56	650305	56
57	650305	57
58	650305	58
59	650305	59
60	650305	60
61	650305	61
62	650305	62
63	650305	63
64	650305	64
65	650305	65
66	650305	66
67	650305	67
68	650305	68
69	650305	69
70	650305	70
71	650305	71
72	650305	72
73	650305	73
74	650305	74
75	650305	75
76	650305	76
77	650305	77
78	650305	78
79	6

569	651673	312
570	651673	313
571	651673	314
572	651673	315
573	651673	316
574	651673	317
575	651673	318
576	651673	319
577	651673	320
578	651673	321
579	651673	322
580	651673	323
581	651673	324
582	651673	325
583	651673	326
584	651673	327
585	651673	328
586	651673	329
587	651673	330
588	651673	331
589	651673	332
590	651673	333
591	651673	334
592	651673	335
593	651673	336
594	651673	337
595	651673	338
596	651673	339
597	651673	340
598	651673	341
599	651673	342
600	651673	343
601	651673	344
602	651673	345
603	651673	346
604	651673	347
605	651673	348
606	651673	349
607	651673	350
608	651673	351
609	651673	352
610	651673	353
611	651673	354
612	651673	355
613	651673	356
614	651673	357
615	651673	358
616	651673	359
617	651673	360
618	651673	361
619	651673	362
620	651673	363
621	651673	364
622	651673	365
623	651673	366
624	651673	367
625	651673	368
626	651673	369
627	651673	370
628	651673	371
629	651673	372
630	651673	373
631	651673	374
632	651673	375
633	651673	376
634	651673	377
635	651673

1119	727188	24
1120	727188	25
1121	727188	26
1122	727188	27
1123	727188	28
1124	727188	29
1125	727188	30
1126	727188	31
1127	727188	32
1128	727188	33
1129	727188	34
1130	727188	35
1131	727188	36
1132	727188	37
1133	727188	38
1134	727188	39
1135	727188	40
1136	727188	41
1137	727188	42
1138	727188	43
1139	727188	44
1140	727188	45
1141	727188	46
1142	727188	47
1143	727188	48
1144	727188	49
1145	727188	50
1146	727188	51
1147	727188	52
1148	727188	53
1149	727188	54
1150	703843	1
1151	703843	2
1152	703843	3
1153	703843	4
1154	703843	5
1155	703843	6
1156	703843	7
1157	703843	8
1158	703843	9
1159	703843	10
1160	703843	11
1161	703843	12
1162	703843	13
1163	703843	14
1164	703843	15
1165	703843	16
1166	703843	17
1167	703843	18
1168	703843	19
1169	703843	20
1170	703843	21
1171	703843	22
1172	703843	23
1173	703843	24
1174	703843	25
1175	703843	26
1176	703843	27
1177	703843	28
1178	703843	29
1179	703843	30
1180	703843	31
1181	703843	32
1182	703843	33
1183	703843	34
1184	703843	35
1185	703843	36
1186

1674	64997	352
1675	64997	353
1676	64997	354
1677	64997	355
1678	64997	356
1679	64997	357
1680	64997	358
1681	64997	359
1682	64997	360
1683	64997	361
1684	64997	362
1685	64997	363
1686	64997	364
1687	64997	365
1688	64997	366
1689	64997	367
1690	64997	368
1691	64997	369
1692	64997	370
1693	64997	371
1694	64997	372
1695	64997	373
1696	64997	374
1697	64997	375
1698	64997	376
1699	64997	377
1700	64997	378
1701	64997	379
1702	64997	380
1703	64997	381
1704	64997	382
1705	64997	383
1706	64997	384
1707	64997	385
1708	64997	386
1709	64997	387
1710	64997	388
1711	64997	389
1712	64997	390
1713	64997	391
1714	64997	392
1715	64997	393
1716	64997	394
1717	64997	395
1718	64997	396
1719	64997	397
1720	64997	398
1721	64997	399
1722	64997	400
1723	64997	401
1724	64997	402
1725	64997	403
1726	64997	404
1727	64997	405
1728	64997	406
1729	64997	407
1730	64997	408
1731	64997	409
1732	64997	410
1733	64997	411
1734	64997	412
1735	64997	413
1736	64997	414
1737	64997	415
1738	64997	416
1739	64997	417
1740	64997

2204	616239	365
2205	616239	366
2206	616239	367
2207	616239	368
2208	616239	369
2209	616239	370
2210	616239	371
2211	616239	372
2212	616239	373
2213	616239	374
2214	616239	375
2215	616239	376
2216	616239	377
2217	616239	378
2218	616239	379
2219	616239	380
2220	616239	381
2221	616239	382
2222	616239	383
2223	616239	384
2224	616239	385
2225	616239	386
2226	616239	387
2227	616239	388
2228	616239	389
2229	616239	390
2230	616239	391
2231	616239	392
2232	616239	393
2233	616239	394
2234	616239	395
2235	616239	396
2236	616239	397
2237	616239	398
2238	616239	399
2239	616239	400
2240	616239	401
2241	616239	402
2242	616239	403
2243	616239	404
2244	616239	405
2245	616239	406
2246	616239	407
2247	616239	408
2248	616239	409
2249	616239	410
2250	616239	411
2251	616239	412
2252	616239	413
2253	616239	414
2254	616239	415
2255	616239	416
2256	616239	417
2257	616239	418
2258	616239	419
2259	616239	420
2260	616239	421
2261	616239	422
2262	616239	423
2263	616239	424
2264	616239	425
2265	616239	426
2266	616

2734	737377	6
2735	737377	7
2736	737377	8
2737	737377	9
2738	737377	10
2739	737377	11
2740	737377	12
2741	737377	13
2742	737377	14
2743	737377	15
2744	737377	16
2745	737377	17
2746	737377	18
2747	737377	19
2748	737377	20
2749	715159	1
2750	715159	2
2751	715159	3
2752	715159	4
2753	715159	5
2754	715159	6
2755	715159	7
2756	715159	8
2757	715159	9
2758	715159	10
2759	715159	11
2760	715159	12
2761	715159	13
2762	715159	14
2763	715159	15
2764	715159	16
2765	715159	17
2766	715159	18
2767	715159	19
2768	715159	20
2769	715159	21
2770	715159	22
2771	715159	23
2772	715159	24
2773	715159	25
2774	715159	26
2775	715159	27
2776	715159	28
2777	715159	29
2778	715159	30
2779	715159	31
2780	715159	32
2781	715159	33
2782	715159	34
2783	715159	35
2784	715159	36
2785	715159	37
2786	715159	38
2787	715159	39
2788	715159	40
2789	715159	41
2790	715159	42
2791	715159	43
2792	715159	44
2793	715159	45
2794	715159	46
2795	715159	47
2796	715159	48
2797	715159	49
2798	715159	50
2799	715159	51
2800	715159	52
2801	715

3286	729047	5
3287	729047	6
3288	729047	7
3289	729047	8
3290	729047	9
3291	729047	10
3292	729047	11
3293	729047	12
3294	729047	13
3295	729047	14
3296	729047	15
3297	729047	16
3298	729047	17
3299	729047	18
3300	729047	19
3301	729047	20
3302	729047	21
3303	729047	22
3304	729047	23
3305	729047	24
3306	729047	25
3307	729047	26
3308	729047	27
3309	729047	28
3310	729047	29
3311	729047	30
3312	729047	31
3313	729047	32
3314	729047	33
3315	729047	34
3316	729047	35
3317	729047	36
3318	729047	37
3319	729047	38
3320	729047	39
3321	729047	40
3322	729047	41
3323	729047	42
3324	729047	43
3325	725552	1
3326	725552	2
3327	725552	3
3328	725552	4
3329	725552	5
3330	725552	6
3331	725552	7
3332	725552	8
3333	725552	9
3334	725552	10
3335	725552	11
3336	725552	12
3337	725552	13
3338	725552	14
3339	725552	15
3340	725552	16
3341	725552	17
3342	725552	18
3343	725552	19
3344	725552	20
3345	725552	21
3346	725552	22
3347	725552	23
3348	725552	24
3349	725552	25
3350	725552	26
3351	725552	27
3352	725552	28
3353	7255

3842	721919	22
3843	721919	23
3844	721919	24
3845	721919	25
3846	721919	26
3847	721919	27
3848	721919	28
3849	721919	29
3850	721919	30
3851	721919	31
3852	721919	32
3853	721919	33
3854	721919	34
3855	721919	35
3856	721919	36
3857	721919	37
3858	721919	38
3859	721919	39
3860	721919	40
3861	721919	41
3862	721919	42
3863	721919	43
3864	721919	44
3865	721919	45
3866	721919	46
3867	721919	47
3868	721919	48
3869	721919	49
3870	721919	50
3871	721919	51
3872	721919	52
3873	721919	53
3874	721919	54
3875	721919	55
3876	721919	56
3877	721919	57
3878	721919	58
3879	721919	59
3880	721919	60
3881	721919	61
3882	721919	62
3883	721919	63
3884	721919	64
3885	721919	65
3886	721919	66
3887	721919	67
3888	721919	68
3889	721919	69
3890	729084	1
3891	729084	2
3892	729084	3
3893	729084	4
3894	729084	5
3895	729084	6
3896	729084	7
3897	729084	8
3898	729084	9
3899	729084	10
3900	729084	11
3901	729084	12
3902	729084	13
3903	729084	14
3904	729084	15
3905	729084	16
3906	729084	17
3907	729084	18
3908	729084	19
3909

In [11]:
pd.DataFrame(comments6).to_csv('./comments/naver_under18_sat.csv', index=False, encoding='utf-8')

In [12]:
start = time.time()
comments3 = get_naver_comment(links3)
pd.DataFrame(comments3).to_csv('./comments/naver_under18_wed.csv', index=False, encoding='utf-8')
print(f'time : {time.time() - start}')

1	662774	1
2	662774	2
3	662774	3
4	662774	4
5	662774	5
6	662774	6
7	662774	7
8	662774	8
9	662774	9
10	662774	10
11	662774	11
12	662774	12
13	662774	13
14	662774	14
15	662774	15
16	662774	16
17	662774	17
18	662774	18
19	662774	19
20	662774	20
21	662774	21
22	662774	22
23	662774	23
24	662774	24
25	662774	25
26	662774	26
27	662774	27
28	662774	28
29	662774	29
30	662774	30
31	662774	31
32	662774	32
33	662774	33
34	662774	34
35	662774	35
36	662774	36
37	662774	37
38	662774	38
39	662774	39
40	662774	40
41	662774	41
42	662774	42
43	662774	43
44	662774	44
45	662774	45
46	662774	46
47	662774	47
48	662774	48
49	662774	49
50	662774	50
51	662774	51
52	662774	52
53	662774	53
54	662774	54
55	662774	55
56	662774	56
57	662774	57
58	662774	58
59	662774	59
60	662774	60
61	662774	61
62	662774	62
63	662774	63
64	662774	64
65	662774	65
66	662774	66
67	662774	67
68	662774	68
69	662774	69
70	662774	70
71	662774	71
72	662774	72
73	662774	73
74	662774	74
75	662774	75
76	662774	76
77	662774	77
78	662774	78
79	6

576	667573	92
577	667573	93
578	667573	94
579	667573	95
580	667573	96
581	667573	97
582	667573	98
583	667573	99
584	667573	100
585	667573	101
586	667573	102
587	667573	103
588	667573	104
589	667573	105
590	667573	106
591	667573	107
592	667573	108
593	667573	109
594	667573	110
595	667573	111
596	667573	112
597	667573	113
598	667573	114
599	667573	115
600	667573	116
601	667573	117
602	667573	118
603	667573	119
604	667573	120
605	667573	121
606	667573	122
607	667573	123
608	667573	124
609	667573	125
610	667573	126
611	667573	127
612	667573	128
613	667573	129
614	667573	130
615	667573	131
616	667573	132
617	667573	133
618	667573	134
619	667573	135
620	667573	136
621	667573	137
622	667573	138
623	667573	139
624	667573	140
625	667573	141
626	667573	142
627	667573	143
628	667573	144
629	667573	145
630	667573	146
631	667573	147
632	667573	148
633	667573	149
634	667573	150
635	667573	151
636	667573	152
637	667573	153
638	667573	154
639	667573	155
640	667573	156
641	667573	157
642	667573	158
643

1136	603159	184
1137	603159	185
1138	603159	186
1139	603159	187
1140	603159	188
1141	603159	189
1142	603159	190
1143	603159	191
1144	603159	192
1145	603159	193
1146	603159	194
1147	603159	195
1148	603159	196
1149	603159	197
1150	603159	198
1151	603159	199
1152	603159	200
1153	603159	201
1154	603159	202
1155	603159	203
1156	603159	204
1157	603159	205
1158	603159	206
1159	603159	207
1160	721457	1
1161	721457	2
1162	721457	3
1163	721457	4
1164	721457	5
1165	721457	6
1166	721457	7
1167	721457	8
1168	721457	9
1169	721457	10
1170	721457	11
1171	721457	12
1172	721457	13
1173	721457	14
1174	721457	15
1175	721457	16
1176	721457	17
1177	721457	18
1178	721457	19
1179	721457	20
1180	721457	21
1181	721457	22
1182	721457	23
1183	721457	24
1184	721457	25
1185	721457	26
1186	721457	27
1187	721457	28
1188	721457	29
1189	721457	30
1190	721457	31
1191	721457	32
1192	721457	33
1193	721457	34
1194	721457	35
1195	721457	36
1196	721457	37
1197	721457	38
1198	721457	39
1199	721457	40
1200	721457	41
1201	72145

1676	717481	33
1677	717481	34
1678	717481	35
1679	717481	36
1680	717481	37
1681	717481	38
1682	717481	39
1683	717481	40
1684	717481	41
1685	717481	42
1686	717481	43
1687	717481	44
1688	717481	45
1689	717481	46
1690	717481	47
1691	717481	48
1692	717481	49
1693	717481	50
1694	717481	51
1695	717481	52
1696	717481	53
1697	717481	54
1698	717481	55
1699	717481	56
1700	717481	57
1701	717481	58
1702	717481	59
1703	717481	60
1704	717481	61
1705	717481	62
1706	717481	63
1707	717481	64
1708	717481	65
1709	717481	66
1710	717481	67
1711	717481	68
1712	717481	69
1713	717481	70
1714	717481	71
1715	717481	72
1716	717481	73
1717	717481	74
1718	717481	75
1719	717481	76
1720	717481	77
1721	717481	78
1722	717481	79
1723	717481	80
1724	717481	81
1725	717481	82
1726	717481	83
1727	738143	1
1728	738143	2
1729	738143	3
1730	738143	4
1731	738143	5
1732	738143	6
1733	738143	7
1734	738143	8
1735	738143	9
1736	738143	10
1737	738143	11
1738	738143	12
1739	738143	13
1740	738143	14
1741	738143	15
1742	738143	16
1743

2214	731297	2
2215	731297	3
2216	731297	4
2217	731297	5
2218	731297	6
2219	731297	7
2220	731297	8
2221	731297	9
2222	731297	10
2223	731297	11
2224	731297	12
2225	731297	13
2226	731297	14
2227	731297	15
2228	731297	16
2229	731297	17
2230	731297	18
2231	731297	19
2232	731297	20
2233	731297	21
2234	731297	22
2235	731297	23
2236	731297	24
2237	731297	25
2238	731297	26
2239	731297	27
2240	731297	28
2241	731297	29
2242	731297	30
2243	731297	31
2244	731297	32
2245	731297	33
2246	731297	34
2247	731297	35
2248	731297	36
2249	731297	37
2250	731297	38
2251	731297	39
2252	731297	40
2253	731297	41
2254	735979	1
2255	735979	2
2256	735979	3
2257	735979	4
2258	735979	5
2259	735979	6
2260	735979	7
2261	735979	8
2262	735979	9
2263	735979	10
2264	735979	11
2265	735979	12
2266	735979	13
2267	735979	14
2268	735979	15
2269	735979	16
2270	735979	17
2271	735979	18
2272	735979	19
2273	735979	20
2274	735979	21
2275	735979	22
2276	735979	23
2277	735979	24
2278	735979	25
2279	735979	26
2280	735979	27
2281	733034	

2753	602916	250
2754	602916	251
2755	602916	252
2756	602916	253
2757	602916	254
2758	602916	255
2759	602916	256
2760	602916	257
2761	602916	258
2762	602916	259
2763	738694	1
2764	741150	1
2765	741150	2
2766	741150	3
2767	741150	4
2768	741150	5
2769	741150	6
2770	741150	7
2771	741150	8
2772	741150	9
2773	741150	10
2774	741150	11
2775	740481	1
2776	740481	2
2777	740481	3
2778	740481	4
2779	740481	5
2780	740481	6
2781	722386	1
2782	722386	2
2783	722386	3
2784	722386	4
2785	722386	5
2786	722386	6
2787	722386	7
2788	722386	8
2789	722386	9
2790	722386	10
2791	722386	11
2792	722386	12
2793	722386	13
2794	722386	14
2795	722386	15
2796	722386	16
2797	722386	17
2798	722386	18
2799	722386	19
2800	722386	20
2801	722386	21
2802	722386	22
2803	722386	23
2804	722386	24
2805	722386	25
2806	722386	26
2807	722386	27
2808	722386	28
2809	722386	29
2810	722386	30
2811	722386	31
2812	722386	32
2813	722386	33
2814	722386	34
2815	722386	35
2816	722386	36
2817	722386	37
2818	722386	38
2819	722386	39
2820	72238

3305	737009	7
3306	737009	8
3307	737009	9
3308	737009	10
3309	737009	11
3310	737009	12
3311	737009	13
3312	737009	14
3313	737009	15
3314	737009	16
3315	737009	17
3316	737009	18
3317	737009	19
3318	737009	20
3319	737009	21
3320	743026	1
3321	743026	2
3322	736699	1
3323	736699	2
3324	736699	3
3325	736699	4
3326	736699	5
3327	736699	6
3328	736699	7
3329	736699	8
3330	736699	9
3331	736699	10
3332	736699	11
3333	736699	12
3334	736699	13
3335	736699	14
3336	736699	15
3337	736699	16
3338	736699	17
3339	736699	18
3340	736699	19
3341	736699	20
3342	736699	21
3343	736699	22
3344	736699	23
3345	736699	24
time : 6891.802193164825


In [14]:
start = time.time()
comments4 = get_naver_comment(links4)
pd.DataFrame(comments4).to_csv('./comments/naver_under18_thu.csv', index=False, encoding='utf-8')
print(f'time : {(time.time() - start) / 60}')

1	570503	1
2	570503	2
3	570503	3
4	570503	4
5	570503	5
6	570503	6
7	570503	7
8	570503	8
9	570503	9
10	570503	10
11	570503	11
12	570503	12
13	570503	13
14	570503	14
15	570503	15
16	570503	16
17	570503	17
18	570503	18
19	570503	19
20	570503	20
21	570503	21
22	570503	22
23	570503	23
24	570503	24
25	570503	25
26	570503	26
27	570503	27
28	570503	28
29	570503	29
30	570503	30
31	570503	31
32	570503	32
33	570503	33
34	570503	34
35	570503	35
36	570503	36
37	570503	37
38	570503	38
39	570503	39
40	570503	40
41	570503	41
42	570503	42
43	570503	43
44	570503	44
45	570503	45
46	570503	46
47	570503	47
48	570503	48
49	570503	49
50	570503	50
51	570503	51
52	570503	52
53	570503	53
54	570503	54
55	570503	55
56	570503	56
57	570503	57
58	570503	58
59	570503	59
60	570503	60
61	570503	61
62	570503	62
63	570503	63
64	570503	64
65	570503	65
66	570503	66
67	570503	67
68	570503	68
69	570503	69
70	570503	70
71	570503	71
72	570503	72
73	570503	73
74	570503	74
75	570503	75
76	570503	76
77	570503	77
78	570503	78
79	5

578	570506	169
579	570506	170
580	570506	171
581	570506	172
582	570506	173
583	570506	174
584	570506	175
585	570506	176
586	570506	177
587	570506	178
588	570506	179
589	570506	180
590	570506	181
591	570506	182
592	570506	183
593	570506	184
594	570506	185
595	570506	186
596	570506	187
597	570506	188
598	570506	189
599	570506	190
600	570506	191
601	570506	192
602	570506	193
603	570506	194
604	570506	195
605	570506	196
606	570506	197
607	570506	198
608	570506	199
609	570506	200
610	570506	201
611	570506	202
612	570506	203
613	570506	204
614	570506	205
615	570506	206
616	570506	207
617	570506	208
618	570506	209
619	570506	210
620	570506	211
621	570506	212
622	570506	213
623	570506	214
624	570506	215
625	570506	216
626	570506	217
627	570506	218
628	570506	219
629	570506	220
630	570506	221
631	570506	222
632	570506	223
633	570506	224
634	570506	225
635	570506	226
636	570506	227
637	570506	228
638	570506	229
639	570506	230
640	570506	231
641	570506	232
642	570506	233
643	570506	234
644	570506

1134	699415	113
1135	699415	114
1136	699415	115
1137	699415	116
1138	699415	117
1139	699415	118
1140	699415	119
1141	699415	120
1142	699415	121
1143	699415	122
1144	699415	123
1145	699415	124
1146	699415	125
1147	699415	126
1148	699415	127
1149	699415	128
1150	699415	129
1151	699415	130
1152	699415	131
1153	699415	132
1154	699415	133
1155	699415	134
1156	699415	135
1157	699415	136
1158	699415	137
1159	699415	138
1160	699415	139
1161	699415	140
1162	699415	141
1163	699415	142
1164	699415	143
1165	699415	144
1166	131385	1
1167	131385	2
1168	131385	3
1169	131385	4
1170	131385	5
1171	131385	6
1172	131385	7
1173	131385	8
1174	131385	9
1175	131385	10
1176	131385	11
1177	131385	12
1178	131385	13
1179	131385	14
1180	131385	15
1181	131385	16
1182	131385	17
1183	131385	18
1184	131385	19
1185	131385	20
1186	131385	21
1187	131385	22
1188	131385	23
1189	131385	24
1190	131385	25
1191	131385	26
1192	131385	27
1193	131385	28
1194	131385	29
1195	131385	30
1196	131385	31
1197	131385	32
1198	131385	33
11

1657	670144	24
1658	670144	25
1659	670144	26
1660	670144	27
1661	670144	28
1662	670144	29
1663	670144	30
1664	670144	31
1665	670144	32
1666	670144	33
1667	670144	34
1668	670144	35
1669	670144	36
1670	670144	37
1671	670144	38
1672	670144	39
1673	670144	40
1674	670144	41
1675	670144	42
1676	670144	43
1677	670144	44
1678	670144	45
1679	670144	46
1680	670144	47
1681	670144	48
1682	670144	49
1683	670144	50
1684	670144	51
1685	670144	52
1686	670144	53
1687	670144	54
1688	670144	55
1689	670144	56
1690	670144	57
1691	670144	58
1692	670144	59
1693	670144	60
1694	670144	61
1695	670144	62
1696	670144	63
1697	670144	64
1698	670144	65
1699	670144	66
1700	670144	67
1701	670144	68
1702	670144	69
1703	670144	70
1704	670144	71
1705	670144	72
1706	670144	73
1707	670144	74
1708	670144	75
1709	670144	76
1710	670144	77
1711	670144	78
1712	670144	79
1713	670144	80
1714	670144	81
1715	670144	82
1716	670144	83
1717	670144	84
1718	670144	85
1719	670144	86
1720	670144	87
1721	670144	88
1722	670144	89
1723	67014

2201	160469	30
2202	160469	31
2203	160469	32
2204	160469	33
2205	160469	34
2206	160469	35
2207	160469	36
2208	160469	37
2209	160469	38
2210	160469	39
2211	160469	40
2212	160469	41
2213	160469	42
2214	160469	43
2215	160469	44
2216	160469	45
2217	160469	46
2218	160469	47
2219	160469	48
2220	160469	49
2221	160469	50
2222	160469	51
2223	160469	52
2224	160469	53
2225	160469	54
2226	160469	55
2227	160469	56
2228	160469	57
2229	160469	58
2230	160469	59
2231	160469	60
2232	160469	61
2233	160469	62
2234	160469	63
2235	160469	64
2236	160469	65
2237	160469	66
2238	160469	67
2239	160469	68
2240	160469	69
2241	160469	70
2242	160469	71
2243	160469	72
2244	160469	73
2245	160469	74
2246	160469	75
2247	160469	76
2248	160469	77
2249	160469	78
2250	160469	79
2251	160469	80
2252	160469	81
2253	160469	82
2254	160469	83
2255	160469	84
2256	160469	85
2257	160469	86
2258	160469	87
2259	160469	88
2260	160469	89
2261	160469	90
2262	160469	91
2263	160469	92
2264	160469	93
2265	160469	94
2266	160469	95
2267	16046

2732	670145	67
2733	670145	68
2734	670145	69
2735	670145	70
2736	670145	71
2737	670145	72
2738	670145	73
2739	670145	74
2740	670145	75
2741	670145	76
2742	670145	77
2743	670145	78
2744	670145	79
2745	670145	80
2746	670145	81
2747	670145	82
2748	670145	83
2749	670145	84
2750	670145	85
2751	670145	86
2752	670145	87
2753	670145	88
2754	670145	89
2755	670145	90
2756	670145	91
2757	670145	92
2758	670145	93
2759	670145	94
2760	670145	95
2761	670145	96
2762	670145	97
2763	670145	98
2764	670145	99
2765	670145	100
2766	670145	101
2767	670145	102
2768	670145	103
2769	670145	104
2770	670145	105
2771	670145	106
2772	670145	107
2773	670145	108
2774	670145	109
2775	670145	110
2776	670145	111
2777	670145	112
2778	670145	113
2779	670145	114
2780	670145	115
2781	670145	116
2782	670145	117
2783	670145	118
2784	670145	119
2785	670145	120
2786	670145	121
2787	670145	122
2788	670145	123
2789	670145	124
2790	670145	125
2791	670145	126
2792	670145	127
2793	670145	128
2794	670145	129
2795	670145	130
2796	6701

3277	733077	30
3278	733077	31
3279	733077	32
3280	733076	1
3281	733076	2
3282	733076	3
3283	733076	4
3284	733076	5
3285	733076	6
3286	733076	7
3287	733076	8
3288	733076	9
3289	733076	10
3290	733076	11
3291	733076	12
3292	733076	13
3293	733076	14
3294	733076	15
3295	733076	16
3296	733076	17
3297	733076	18
3298	733076	19
3299	733076	20
3300	733076	21
3301	733076	22
3302	733076	23
3303	733076	24
3304	733076	25
3305	733076	26
3306	733076	27
3307	733076	28
3308	733076	29
3309	733076	30
3310	733076	31
3311	733076	32
3312	733076	33
3313	733076	34
3314	742048	1
3315	742048	2
3316	742048	3
3317	742048	4
3318	742048	5
3319	742048	6
3320	742048	7
3321	729086	1
3322	729086	2
3323	729086	3
3324	729086	4
3325	729086	5
3326	729086	6
3327	729086	7
3328	729086	8
3329	729086	9
3330	729086	10
3331	729086	11
3332	729086	12
3333	729086	13
3334	729086	14
3335	729086	15
3336	729086	16
3337	729086	17
3338	729086	18
3339	729086	19
3340	729086	20
3341	729086	21
3342	729086	22
3343	729086	23
3344	729086	24
3345	

In [15]:
start = time.time()
comments1 = get_naver_comment(links1)
pd.DataFrame(comments1).to_csv('./comments/naver_under18_mon.csv', index=False, encoding='utf-8')
print(f'time : {(time.time() - start) / 60}')

1	183559	1
2	183559	2
3	183559	3
4	183559	4
5	183559	5
6	183559	6
7	183559	7
8	183559	8
9	183559	9
10	183559	10
11	183559	11
12	183559	12
13	183559	13
14	183559	14
15	183559	15
16	183559	16
17	183559	17
18	183559	18
19	183559	19
20	183559	20
21	183559	21
22	183559	22
23	183559	23
24	183559	24
25	183559	25
26	183559	26
27	183559	27
28	183559	28
29	183559	29
30	183559	30
31	183559	31
32	183559	32
33	183559	33
34	183559	34
35	183559	35
36	183559	36
37	183559	37
38	183559	38
39	183559	39
40	183559	40
41	183559	41
42	183559	42
43	183559	43
44	183559	44
45	183559	45
46	183559	46
47	183559	47
48	183559	48
49	183559	49
50	183559	50
51	183559	51
52	183559	52
53	183559	53
54	183559	54
55	183559	55
56	183559	56
57	183559	57
58	183559	58
59	183559	59
60	183559	60
61	183559	61
62	183559	62
63	183559	63
64	183559	64
65	183559	65
66	183559	66
67	183559	67
68	183559	68
69	183559	69
70	183559	70
71	183559	71
72	183559	72
73	183559	73
74	183559	74
75	183559	75
76	183559	76
77	183559	77
78	183559	78
79	1

569	602910	90
570	602910	91
571	602910	92
572	602910	93
573	602910	94
574	602910	95
575	602910	96
576	602910	97
577	602910	98
578	602910	99
579	602910	100
580	602910	101
581	602910	102
582	602910	103
583	602910	104
584	602910	105
585	602910	106
586	602910	107
587	602910	108
588	602910	109
589	602910	110
590	602910	111
591	602910	112
592	602910	113
593	602910	114
594	602910	115
595	602910	116
596	602910	117
597	602910	118
598	602910	119
599	602910	120
600	602910	121
601	602910	122
602	602910	123
603	602910	124
604	602910	125
605	602910	126
606	602910	127
607	602910	128
608	602910	129
609	602910	130
610	602910	131
611	602910	132
612	602910	133
613	602910	134
614	602910	135
615	602910	136
616	602910	137
617	602910	138
618	602910	139
619	602910	140
620	602910	141
621	602910	142
622	602910	143
623	602910	144
624	602910	145
625	602910	146
626	602910	147
627	602910	148
628	602910	149
629	602910	150
630	602910	151
631	602910	152
632	602910	153
633	602910	154
634	602910	155
635	602910	156
636	6

1124	597478	96
1125	597478	97
1126	597478	98
1127	597478	99
1128	597478	100
1129	597478	101
1130	597478	102
1131	597478	103
1132	597478	104
1133	597478	105
1134	597478	106
1135	597478	107
1136	597478	108
1137	597478	109
1138	597478	110
1139	597478	111
1140	597478	112
1141	597478	113
1142	597478	114
1143	597478	115
1144	597478	116
1145	597478	117
1146	597478	118
1147	597478	119
1148	597478	120
1149	597478	121
1150	597478	122
1151	597478	123
1152	597478	124
1153	597478	125
1154	597478	126
1155	597478	127
1156	597478	128
1157	597478	129
1158	597478	130
1159	597478	131
1160	597478	132
1161	597478	133
1162	597478	134
1163	597478	135
1164	597478	136
1165	597478	137
1166	597478	138
1167	597478	139
1168	597478	140
1169	597478	141
1170	597478	142
1171	597478	143
1172	597478	144
1173	597478	145
1174	597478	146
1175	597478	147
1176	597478	148
1177	597478	149
1178	597478	150
1179	597478	151
1180	597478	152
1181	597478	153
1182	597478	154
1183	597478	155
1184	597478	156
1185	597478	157
1186	597478	

1658	730174	29
1659	730174	30
1660	730174	31
1661	730174	32
1662	730174	33
1663	730174	34
1664	730174	35
1665	730174	36
1666	730174	37
1667	730174	38
1668	730174	39
1669	730174	40
1670	730174	41
1671	730174	42
1672	730174	43
1673	730174	44
1674	729946	1
1675	729946	2
1676	729946	3
1677	729946	4
1678	729946	5
1679	729946	6
1680	729946	7
1681	729946	8
1682	729946	9
1683	729946	10
1684	729946	11
1685	729946	12
1686	729946	13
1687	729946	14
1688	729946	15
1689	729946	16
1690	729946	17
1691	729946	18
1692	729946	19
1693	729946	20
1694	729946	21
1695	729946	22
1696	729946	23
1697	729946	24
1698	729946	25
1699	729946	26
1700	729946	27
1701	729946	28
1702	729946	29
1703	729946	30
1704	729946	31
1705	729946	32
1706	729946	33
1707	729946	34
1708	729946	35
1709	729946	36
1710	729946	37
1711	729946	38
1712	729946	39
1713	729946	40
1714	729946	41
1715	729946	42
1716	729946	43
1717	729946	44
1718	729946	45
1719	743025	1
1720	743025	2
1721	743025	3
1722	743025	4
1723	743025	5
1724	713975	1
1725	71397

2215	21815	315
2216	21815	316
2217	21815	317
2218	21815	318
2219	21815	319
2220	21815	320
2221	21815	321
2222	21815	322
2223	21815	323
2224	21815	324
2225	21815	325
2226	21815	326
2227	21815	327
2228	21815	328
2229	21815	329
2230	21815	330
2231	21815	331
2232	21815	332
2233	21815	333
2234	21815	334
2235	21815	335
2236	21815	336
2237	21815	337
2238	21815	338
2239	21815	339
2240	21815	340
2241	21815	341
2242	21815	342
2243	21815	343
2244	21815	344
2245	21815	345
2246	21815	346
2247	21815	347
2248	21815	348
2249	21815	349
2250	21815	350
2251	21815	351
2252	21815	352
2253	21815	353
2254	21815	354
2255	21815	355
2256	21815	356
2257	21815	357
2258	21815	358
2259	21815	359
2260	21815	360
2261	21815	361
2262	21815	362
2263	21815	363
2264	21815	364
2265	21815	365
2266	21815	366
2267	21815	367
2268	21815	368
2269	21815	369
2270	21815	370
2271	21815	371
2272	21815	372
2273	21815	373
2274	21815	374
2275	21815	375
2276	21815	376
2277	21815	377
2278	21815	378
2279	21815	379
2280	21815	380
2281	21815

2763	703849	49
2764	703849	50
2765	703849	51
2766	703849	52
2767	703849	53
2768	703849	54
2769	703849	55
2770	703849	56
2771	703849	57
2772	703849	58
2773	703849	59
2774	703849	60
2775	703849	61
2776	703849	62
2777	703849	63
2778	703849	64
2779	703849	65
2780	703849	66
2781	703849	67
2782	703849	68
2783	703849	69
2784	703849	70
2785	703849	71
2786	703849	72
2787	703849	73
2788	703849	74
2789	703849	75
2790	703849	76
2791	703849	77
2792	703849	78
2793	703849	79
2794	703849	80
2795	703849	81
2796	703849	82
2797	703849	83
2798	703849	84
2799	703849	85
2800	703849	86
2801	703849	87
2802	703849	88
2803	703849	89
2804	703849	90
2805	703849	91
2806	703849	92
2807	703849	93
2808	703849	94
2809	703849	95
2810	703849	96
2811	703849	97
2812	703849	98
2813	698888	1
2814	698888	2
2815	698888	3
2816	698888	4
2817	698888	5
2818	698888	6
2819	698888	7
2820	698888	8
2821	698888	9
2822	698888	10
2823	698888	11
2824	698888	12
2825	698888	13
2826	698888	14
2827	698888	15
2828	698888	16
2829	698888	17
2830

3313	687915	33
3314	687915	34
3315	687915	35
3316	687915	36
3317	687915	37
3318	687915	38
3319	687915	39
3320	687915	40
3321	687915	41
3322	687915	42
3323	687915	43
3324	687915	44
3325	687915	45
3326	687915	46
3327	687915	47
3328	687915	48
3329	687915	49
3330	687915	50
3331	687915	51
3332	687915	52
3333	687915	53
3334	687915	54
3335	687915	55
3336	687915	56
3337	687915	57
3338	687915	58
3339	687915	59
3340	687915	60
3341	687915	61
3342	687915	62
3343	687915	63
3344	687915	64
3345	687915	65
3346	687915	66
3347	687915	67
3348	687915	68
3349	687915	69
3350	687915	70
3351	687915	71
3352	687915	72
3353	687915	73
3354	687915	74
3355	687915	75
3356	687915	76
3357	687915	77
3358	687915	78
3359	687915	79
3360	687915	80
3361	687915	81
3362	687915	82
3363	687915	83
3364	687915	84
3365	687915	85
3366	687915	86
3367	687915	87
3368	687915	88
3369	687915	89
3370	687915	90
3371	687915	91
3372	687915	92
3373	687915	93
3374	687915	94
3375	687915	95
3376	687915	96
3377	687915	97
3378	687915	98
3379	68791

3859	733770	3
3860	733770	4
3861	733770	5
3862	733770	6
3863	733770	7
3864	733770	8
3865	733770	9
3866	733770	10
3867	733770	11
3868	733770	12
3869	733770	13
3870	733770	14
3871	733770	15
3872	733770	16
3873	733770	17
3874	733770	18
3875	733770	19
3876	733770	20
3877	733770	21
3878	733770	22
3879	733770	23
3880	733770	24
3881	733770	25
3882	733770	26
3883	733770	27
3884	733770	28
3885	733770	29
3886	733770	30
3887	733770	31
3888	733770	32
3889	733770	33
3890	721455	1
3891	721455	2
3892	721455	3
3893	721455	4
3894	721455	5
3895	721455	6
3896	721455	7
3897	721455	8
3898	721455	9
3899	721455	10
3900	721455	11
3901	721455	12
3902	721455	13
3903	721455	14
3904	721455	15
3905	721455	16
3906	721455	17
3907	721455	18
3908	721455	19
3909	721455	20
3910	721455	21
3911	721455	22
3912	721455	23
3913	721455	24
3914	721455	25
3915	721455	26
3916	721455	27
3917	721455	28
3918	721455	29
3919	721455	30
3920	721455	31
3921	721455	32
3922	721455	33
3923	721455	34
3924	721455	35
3925	721455	36
3926	721455

In [16]:
start = time.time()
comments5 = get_naver_comment(links5)
pd.DataFrame(comments5).to_csv('./comments/naver_under18_fri.csv', index=False, encoding='utf-8')
print(f'time : {(time.time() - start) / 60}')

1	641253	1
2	641253	2
3	641253	3
4	641253	4
5	641253	5
6	641253	6
7	641253	7
8	641253	8
9	641253	9
10	641253	10
11	641253	11
12	641253	12
13	641253	13
14	641253	14
15	641253	15
16	641253	16
17	641253	17
18	641253	18
19	641253	19
20	641253	20
21	641253	21
22	641253	22
23	641253	23
24	641253	24
25	641253	25
26	641253	26
27	641253	27
28	641253	28
29	641253	29
30	641253	30
31	641253	31
32	641253	32
33	641253	33
34	641253	34
35	641253	35
36	641253	36
37	641253	37
38	641253	38
39	641253	39
40	641253	40
41	641253	41
42	641253	42
43	641253	43
44	641253	44
45	641253	45
46	641253	46
47	641253	47
48	641253	48
49	641253	49
50	641253	50
51	641253	51
52	641253	52
53	641253	53
54	641253	54
55	641253	55
56	641253	56
57	641253	57
58	641253	58
59	641253	59
60	641253	60
61	641253	61
62	641253	62
63	641253	63
64	641253	64
65	641253	65
66	641253	66
67	641253	67
68	641253	68
69	641253	69
70	641253	70
71	641253	71
72	641253	72
73	641253	73
74	641253	74
75	641253	75
76	641253	76
77	641253	77
78	641253	78
79	6

569	318995	284
570	318995	285
571	318995	286
572	318995	287
573	318995	288
574	318995	289
575	318995	290
576	318995	291
577	318995	292
578	318995	293
579	318995	294
580	318995	295
581	318995	296
582	318995	297
583	318995	298
584	318995	299
585	318995	300
586	318995	301
587	318995	302
588	318995	303
589	318995	304
590	318995	305
591	318995	306
592	318995	307
593	318995	308
594	318995	309
595	318995	310
596	318995	311
597	318995	312
598	318995	313
599	318995	314
600	318995	315
601	318995	316
602	318995	317
603	318995	318
604	318995	319
605	318995	320
606	318995	321
607	318995	322
608	318995	323
609	318995	324
610	318995	325
611	318995	326
612	318995	327
613	318995	328
614	318995	329
615	318995	330
616	318995	331
617	318995	332
618	318995	333
619	318995	334
620	318995	335
621	318995	336
622	318995	337
623	318995	338
624	318995	339
625	318995	340
626	318995	341
627	318995	342
628	318995	343
629	318995	344
630	318995	345
631	318995	346
632	318995	347
633	318995	348
634	318995	349
635	318995

1126	552960	189
1127	552960	190
1128	552960	191
1129	552960	192
1130	552960	193
1131	552960	194
1132	552960	195
1133	552960	196
1134	552960	197
1135	552960	198
1136	552960	199
1137	552960	200
1138	552960	201
1139	552960	202
1140	552960	203
1141	552960	204
1142	552960	205
1143	552960	206
1144	552960	207
1145	552960	208
1146	552960	209
1147	552960	210
1148	552960	211
1149	552960	212
1150	552960	213
1151	552960	214
1152	552960	215
1153	552960	216
1154	552960	217
1155	552960	218
1156	552960	219
1157	552960	220
1158	552960	221
1159	552960	222
1160	552960	223
1161	552960	224
1162	552960	225
1163	552960	226
1164	552960	227
1165	552960	228
1166	552960	229
1167	552960	230
1168	552960	231
1169	552960	232
1170	552960	233
1171	552960	234
1172	552960	235
1173	552960	236
1174	552960	237
1175	552960	238
1176	552960	239
1177	552960	240
1178	552960	241
1179	552960	242
1180	552960	243
1181	552960	244
1182	552960	245
1183	552960	246
1184	552960	247
1185	552960	248
1186	552960	249
1187	552960	250
1188	552

1659	710754	27
1660	710754	28
1661	710754	29
1662	710754	30
1663	710754	31
1664	710754	32
1665	710754	33
1666	710754	34
1667	710754	35
1668	710754	36
1669	710754	37
1670	710754	38
1671	710754	39
1672	710754	40
1673	710754	41
1674	710754	42
1675	710754	43
1676	710754	44
1677	710754	45
1678	710754	46
1679	710754	47
1680	710754	48
1681	710754	49
1682	710754	50
1683	710754	51
1684	710754	52
1685	710754	53
1686	710754	54
1687	710754	55
1688	710754	56
1689	710754	57
1690	710754	58
1691	710754	59
1692	710754	60
1693	710754	61
1694	710754	62
1695	710754	63
1696	710754	64
1697	710754	65
1698	710754	66
1699	710754	67
1700	710754	68
1701	710754	69
1702	710754	70
1703	710754	71
1704	710754	72
1705	710754	73
1706	710754	74
1707	710754	75
1708	710754	76
1709	710754	77
1710	710754	78
1711	710754	79
1712	710754	80
1713	710754	81
1714	710754	82
1715	710754	83
1716	710754	84
1717	710754	85
1718	710754	86
1719	710754	87
1720	710754	88
1721	710754	89
1722	710754	90
1723	710754	91
1724	710754	92
1725	71075

2198	524520	151
2199	524520	152
2200	524520	153
2201	524520	154
2202	524520	155
2203	524520	156
2204	524520	157
2205	524520	158
2206	524520	159
2207	524520	160
2208	524520	161
2209	524520	162
2210	524520	163
2211	524520	164
2212	524520	165
2213	524520	166
2214	524520	167
2215	524520	168
2216	524520	169
2217	524520	170
2218	524520	171
2219	524520	172
2220	524520	173
2221	524520	174
2222	524520	175
2223	524520	176
2224	524520	177
2225	524520	178
2226	524520	179
2227	524520	180
2228	524520	181
2229	524520	182
2230	524520	183
2231	524520	184
2232	524520	185
2233	524520	186
2234	524520	187
2235	524520	188
2236	524520	189
2237	524520	190
2238	524520	191
2239	524520	192
2240	524520	193
2241	524520	194
2242	524520	195
2243	524520	196
2244	524520	197
2245	524520	198
2246	524520	199
2247	524520	200
2248	524520	201
2249	524520	202
2250	524520	203
2251	524520	204
2252	524520	205
2253	524520	206
2254	524520	207
2255	524520	208
2256	524520	209
2257	524520	210
2258	524520	211
2259	524520	212
2260	524

2726	734040	26
2727	734040	27
2728	734040	28
2729	734040	29
2730	734040	30
2731	734040	31
2732	734040	32
2733	735243	1
2734	735243	2
2735	735243	3
2736	735243	4
2737	735243	5
2738	735243	6
2739	735243	7
2740	735243	8
2741	735243	9
2742	735243	10
2743	735243	11
2744	735243	12
2745	735243	13
2746	735243	14
2747	735243	15
2748	735243	16
2749	735243	17
2750	735243	18
2751	735243	19
2752	735243	20
2753	735243	21
2754	735243	22
2755	735243	23
2756	735243	24
2757	735243	25
2758	735243	26
2759	735243	27
2760	735243	28
2761	735243	29
2762	718019	1
2763	718019	2
2764	718019	3
2765	718019	4
2766	718019	5
2767	718019	6
2768	718019	7
2769	718019	8
2770	718019	9
2771	718019	10
2772	718019	11
2773	718019	12
2774	718019	13
2775	718019	14
2776	718019	15
2777	718019	16
2778	718019	17
2779	718019	18
2780	718019	19
2781	718019	20
2782	718019	21
2783	718019	22
2784	718019	23
2785	718019	24
2786	718019	25
2787	718019	26
2788	718019	27
2789	718019	28
2790	718019	29
2791	718019	30
2792	718019	31
2793	718019	3

3274	700844	13
3275	700844	14
3276	700844	15
3277	700844	16
3278	700844	17
3279	700844	18
3280	700844	19
3281	700844	20
3282	700844	21
3283	700844	22
3284	700844	23
3285	700844	24
3286	700844	25
3287	700844	26
3288	700844	27
3289	700844	28
3290	700844	29
3291	700844	30
3292	700844	31
3293	700844	32
3294	700844	33
3295	700844	34
3296	700844	35
3297	700844	36
3298	700844	37
3299	700844	38
3300	700844	39
3301	700844	40
3302	700844	41
3303	700844	42
3304	700844	43
3305	700844	44
3306	700844	45
3307	700844	46
3308	700844	47
3309	700844	48
3310	700844	49
3311	700844	50
3312	700844	51
3313	700844	52
3314	700844	53
3315	700844	54
3316	700844	55
3317	700844	56
3318	700844	57
3319	700844	58
3320	700844	59
3321	700844	60
3322	700844	61
3323	700844	62
3324	700844	63
3325	700844	64
3326	700844	65
3327	700844	66
3328	700844	67
3329	700844	68
3330	700844	69
3331	700844	70
3332	700844	71
3333	700844	72
3334	700844	73
3335	700844	74
3336	700844	75
3337	700844	76
3338	700844	77
3339	700844	78
3340	70084

3819	724431	19
3820	724431	20
3821	724431	21
3822	724431	22
3823	724431	23
3824	724431	24
3825	724431	25
3826	724431	26
3827	724431	27
3828	724431	28
3829	724431	29
3830	724431	30
3831	724431	31
3832	724431	32
3833	724431	33
3834	724431	34
3835	724431	35
3836	724431	36
3837	724431	37
3838	724431	38
3839	724431	39
3840	724431	40
3841	724431	41
3842	724431	42
3843	724431	43
3844	724431	44
3845	724431	45
3846	724431	46
3847	724431	47
3848	724431	48
3849	724431	49
3850	724431	50
3851	724431	51
3852	724431	52
3853	724431	53
3854	724431	54
3855	724431	55
3856	724431	56
3857	724431	57
3858	724431	58
3859	724431	59
3860	724431	60
3861	724431	61
3862	724431	62
3863	724431	63
3864	724431	64
3865	724431	65
3866	742349	1
3867	742349	2
3868	742349	3
3869	742349	4
3870	742349	5
3871	742349	6
3872	742349	7
3873	742349	8
3874	739350	1
3875	739350	2
3876	739350	3
3877	739350	4
3878	739350	5
3879	739350	6
3880	739350	7
3881	739350	8
3882	739350	9
3883	739350	10
3884	739350	11
3885	739350	12
3886	729089	

In [17]:
start = time.time()
comments7 = get_naver_comment(links7)
pd.DataFrame(comments7).to_csv('./comments/naver_under18_sun.csv', index=False, encoding='utf-8')
print(f'time : {(time.time() - start) / 60}')

1	736277	1
2	736277	2
3	736277	3
4	736277	4
5	736277	5
6	736277	6
7	736277	7
8	736277	8
9	736277	9
10	736277	10
11	736277	11
12	736277	12
13	736277	13
14	736277	14
15	736277	15
16	736277	16
17	736277	17
18	736277	18
19	736277	19
20	736277	20
21	736277	21
22	736277	22
23	736277	23
24	736277	24
25	670152	1
26	670152	2
27	670152	3
28	670152	4
29	670152	5
30	670152	6
31	670152	7
32	670152	8
33	670152	9
34	670152	10
35	670152	11
36	670152	12
37	670152	13
38	670152	14
39	670152	15
40	670152	16
41	670152	17
42	670152	18
43	670152	19
44	670152	20
45	670152	21
46	670152	22
47	670152	23
48	670152	24
49	670152	25
50	670152	26
51	670152	27
52	670152	28
53	670152	29
54	670152	30
55	670152	31
56	670152	32
57	670152	33
58	670152	34
59	670152	35
60	670152	36
61	670152	37
62	670152	38
63	670152	39
64	670152	40
65	670152	41
66	670152	42
67	670152	43
68	670152	44
69	670152	45
70	670152	46
71	670152	47
72	670152	48
73	670152	49
74	670152	50
75	670152	51
76	670152	52
77	670152	53
78	670152	54
79	670152	55


579	655746	233
580	655746	234
581	655746	235
582	655746	236
583	655746	237
584	655746	238
585	729326	1
586	729326	2
587	729326	3
588	729326	4
589	729326	5
590	729326	6
591	729326	7
592	729326	8
593	729326	9
594	729326	10
595	729326	11
596	729326	12
597	729326	13
598	729326	14
599	729326	15
600	729326	16
601	729326	17
602	729326	18
603	729326	19
604	729326	20
605	729326	21
606	729326	22
607	729326	23
608	729326	24
609	729326	25
610	729326	26
611	729326	27
612	729326	28
613	729326	29
614	729326	30
615	729326	31
616	729326	32
617	729326	33
618	729326	34
619	729326	35
620	729326	36
621	729326	37
622	729326	38
623	729326	39
624	729326	40
625	729326	41
626	729326	42
627	729326	43
628	729326	44
629	729326	45
630	729326	46
631	729326	47
632	557676	1
633	557676	2
634	557676	3
635	557676	4
636	557676	5
637	557676	6
638	557676	7
639	557676	8
640	557676	9
641	557676	10
642	557676	11
643	557676	12
644	557676	13
645	557676	14
646	557676	15
647	557676	16
648	557676	17
649	557676	18
650	557676	19
651	

1136	724854	14
1137	724854	15
1138	724854	16
1139	724854	17
1140	724854	18
1141	724854	19
1142	724854	20
1143	724854	21
1144	724854	22
1145	724854	23
1146	724854	24
1147	724854	25
1148	724854	26
1149	724854	27
1150	724854	28
1151	724854	29
1152	724854	30
1153	724854	31
1154	724854	32
1155	724854	33
1156	724854	34
1157	724854	35
1158	724854	36
1159	724854	37
1160	724854	38
1161	724854	39
1162	724854	40
1163	724854	41
1164	724854	42
1165	724854	43
1166	724854	44
1167	724854	45
1168	724854	46
1169	724854	47
1170	724854	48
1171	724854	49
1172	724854	50
1173	724854	51
1174	724854	52
1175	724854	53
1176	724854	54
1177	724854	55
1178	724854	56
1179	724854	57
1180	724854	58
1181	724854	59
1182	724854	60
1183	724854	61
1184	743838	1
1185	743838	2
1186	743838	3
1187	743838	4
1188	718021	1
1189	718021	2
1190	718021	3
1191	718021	4
1192	718021	5
1193	718021	6
1194	718021	7
1195	718021	8
1196	718021	9
1197	718021	10
1198	718021	11
1199	718021	12
1200	718021	13
1201	718021	14
1202	718021	15
1203	718

1686	22897	256
1687	22897	257
1688	22897	258
1689	22897	259
1690	22897	260
1691	22897	261
1692	22897	262
1693	22897	263
1694	22897	264
1695	22897	265
1696	22897	266
1697	22897	267
1698	22897	268
1699	22897	269
1700	22897	270
1701	22897	271
1702	22897	272
1703	22897	273
1704	22897	274
1705	22897	275
1706	22897	276
1707	22897	277
1708	22897	278
1709	22897	279
1710	22897	280
1711	22897	281
1712	22897	282
1713	22897	283
1714	22897	284
1715	22897	285
1716	22897	286
1717	22897	287
1718	22897	288
1719	22897	289
1720	22897	290
1721	22897	291
1722	22897	292
1723	22897	293
1724	22897	294
1725	22897	295
1726	22897	296
1727	22897	297
1728	22897	298
1729	22897	299
1730	22897	300
1731	22897	301
1732	22897	302
1733	22897	303
1734	22897	304
1735	22897	305
1736	22897	306
1737	22897	307
1738	22897	308
1739	22897	309
1740	22897	310
1741	22897	311
1742	22897	312
1743	22897	313
1744	22897	314
1745	22897	315
1746	22897	316
1747	22897	317
1748	22897	318
1749	22897	319
1750	22897	320
1751	22897	321
1752	22897

2233	695796	107
2234	695796	108
2235	695796	109
2236	695796	110
2237	695796	111
2238	695796	112
2239	695796	113
2240	695796	114
2241	695796	115
2242	695796	116
2243	695796	117
2244	695796	118
2245	695796	119
2246	695796	120
2247	695796	121
2248	695796	122
2249	695796	123
2250	695796	124
2251	695796	125
2252	695796	126
2253	695796	127
2254	695796	128
2255	695796	129
2256	695796	130
2257	695796	131
2258	695796	132
2259	695796	133
2260	695796	134
2261	695796	135
2262	695796	136
2263	695796	137
2264	695796	138
2265	695796	139
2266	695796	140
2267	695796	141
2268	695796	142
2269	695796	143
2270	695796	144
2271	695796	145
2272	695796	146
2273	695796	147
2274	695796	148
2275	695796	149
2276	695796	150
2277	695796	151
2278	695796	152
2279	695796	153
2280	721463	1
2281	721463	2
2282	721463	3
2283	721463	4
2284	721463	5
2285	721463	6
2286	721463	7
2287	721463	8
2288	721463	9
2289	721463	10
2290	721463	11
2291	721463	12
2292	721463	13
2293	721463	14
2294	721463	15
2295	721463	16
2296	721463	17
22

2781	626906	127
2782	626906	128
2783	626906	129
2784	626906	130
2785	626906	131
2786	626906	132
2787	626906	133
2788	626906	134
2789	626906	135
2790	626906	136
2791	626906	137
2792	626906	138
2793	626906	139
2794	626906	140
2795	626906	141
2796	626906	142
2797	626906	143
2798	626906	144
2799	626906	145
2800	626906	146
2801	626906	147
2802	626906	148
2803	626906	149
2804	626906	150
2805	626906	151
2806	626906	152
2807	626906	153
2808	626906	154
2809	626906	155
2810	626906	156
2811	626906	157
2812	626906	158
2813	626906	159
2814	626906	160
2815	626906	161
2816	626906	162
2817	626906	163
2818	626906	164
2819	626906	165
2820	626906	166
2821	626906	167
2822	626906	168
2823	626906	169
2824	626906	170
2825	626906	171
2826	626906	172
2827	626906	173
2828	626906	174
2829	626906	175
2830	626906	176
2831	626906	177
2832	626906	178
2833	626906	179
2834	626906	180
2835	626906	181
2836	626906	182
2837	626906	183
2838	626906	184
2839	626906	185
2840	626906	186
2841	626906	187
2842	626906	188
2843	626

3299	732955	19
3300	732955	20
3301	732955	21
3302	732955	22
3303	732955	23
3304	732955	24
3305	732955	25
3306	732955	26
3307	732955	27
3308	732955	28
3309	732955	29
3310	732955	30
3311	732955	31
3312	732955	32
3313	732955	33
3314	732955	34
3315	732955	35
3316	732955	36
3317	732955	37
3318	732955	38
3319	732955	39
3320	732955	40
3321	732955	41
3322	732955	42
3323	732955	43
3324	732955	44
3325	732955	45
3326	732955	46
3327	732955	47
3328	732955	48
3329	732955	49
3330	732955	50
3331	732955	51
3332	732955	52
3333	732955	53
3334	732955	54
3335	732955	55
3336	732955	56
3337	732955	57
3338	732955	58
3339	732955	59
3340	732955	60
3341	732955	61
3342	732955	62
3343	732955	63
3344	732955	64
3345	732955	65
3346	732955	66
3347	732955	67
3348	732955	68
3349	732955	69
3350	732955	70
3351	732955	71
3352	732224	1
3353	732224	2
3354	732224	3
3355	732224	4
3356	732224	5
3357	732224	6
3358	732224	7
3359	732224	8
3360	732224	9
3361	732224	10
3362	732224	11
3363	732224	12
3364	732224	13
3365	732224	14
3366

3847	737021	15
3848	737021	16
3849	737021	17
3850	737021	18
3851	721109	1
3852	721109	2
3853	721109	3
3854	721109	4
3855	721109	5
3856	721109	6
3857	721109	7
3858	721109	8
3859	721109	9
3860	721109	10
3861	721109	11
3862	721109	12
3863	721109	13
3864	721109	14
3865	721109	15
3866	721109	16
3867	721109	17
3868	721109	18
3869	721109	19
3870	721109	20
3871	721109	21
3872	721109	22
3873	721109	23
3874	721109	24
3875	721109	25
3876	721109	26
3877	721109	27
3878	721109	28
3879	721109	29
3880	721109	30
3881	721109	31
3882	721109	32
3883	721109	33
3884	721109	34
3885	721109	35
3886	721109	36
3887	721109	37
3888	721109	38
3889	721109	39
3890	721109	40
3891	721109	41
3892	721109	42
3893	721109	43
3894	721109	44
3895	721109	45
3896	721109	46
3897	721109	47
3898	721109	48
3899	721109	49
3900	721109	50
3901	721109	51
3902	721109	52
3903	721109	53
3904	721109	54
3905	721109	55
3906	721109	56
3907	721109	57
3908	721109	58
3909	721109	59
3910	721109	60
3911	721109	61
3912	721109	62
3913	721109	63
3914

4399	713055	93
4400	713055	94
4401	713055	95
4402	713055	96
4403	713055	97
time : 144.39559326569238


In [18]:
start = time.time()
comments2 = get_naver_comment(links2)
pd.DataFrame(comments2).to_csv('./comments/naver_under18_tue.csv', index=False, encoding='utf-8')
print(f'time : {(time.time() - start) / 60}')

1	703846	1
2	703846	2
3	703846	3
4	703846	4
5	703846	5
6	703846	6
7	703846	7
8	703846	8
9	703846	9
10	703846	10
11	703846	11
12	703846	12
13	703846	13
14	703846	14
15	703846	15
16	703846	16
17	703846	17
18	703846	18
19	703846	19
20	703846	20
21	703846	21
22	703846	22
23	703846	23
24	703846	24
25	703846	25
26	703846	26
27	703846	27
28	703846	28
29	703846	29
30	703846	30
31	703846	31
32	703846	32
33	703846	33
34	703846	34
35	703846	35
36	703846	36
37	703846	37
38	703846	38
39	703846	39
40	703846	40
41	703846	41
42	703846	42
43	703846	43
44	703846	44
45	703846	45
46	703846	46
47	703846	47
48	703846	48
49	703846	49
50	703846	50
51	703846	51
52	703846	52
53	703846	53
54	703846	54
55	703846	55
56	703846	56
57	703846	57
58	703846	58
59	703846	59
60	703846	60
61	703846	61
62	703846	62
63	703846	63
64	703846	64
65	703846	65
66	703846	66
67	703846	67
68	703846	68
69	703846	69
70	703846	70
71	703846	71
72	703846	72
73	703846	73
74	703846	74
75	703846	75
76	703846	76
77	703846	77
78	703846	78
79	7

602	20853	494
603	20853	495
604	20853	496
605	20853	497
606	20853	498
607	20853	499
608	20853	500
609	20853	501
610	20853	502
611	20853	503
612	20853	504
613	20853	505
614	20853	506
615	20853	507
616	20853	508
617	20853	509
618	20853	510
619	20853	511
620	20853	512
621	20853	513
622	20853	514
623	20853	515
624	20853	516
625	20853	517
626	20853	518
627	20853	519
628	20853	520
629	20853	521
630	20853	522
631	20853	523
632	20853	524
633	20853	525
634	20853	526
635	20853	527
636	20853	528
637	20853	529
638	20853	530
639	20853	531
640	20853	532
641	20853	533
642	20853	534
643	20853	535
644	20853	536
645	20853	537
646	20853	538
647	20853	539
648	20853	540
649	20853	541
650	20853	542
651	20853	543
652	20853	544
653	20853	545
654	20853	546
655	20853	547
656	20853	548
657	20853	549
658	20853	550
659	20853	551
660	20853	552
661	20853	553
662	20853	554
663	20853	555
664	20853	556
665	20853	557
666	20853	558
667	20853	559
668	20853	560
669	20853	561
670	20853	562
671	20853	563
672	20853	564
673	20

1171	20853	1063
1172	20853	1064
1173	20853	1065
1174	20853	1066
1175	20853	1067
1176	20853	1068
1177	20853	1069
1178	20853	1070
1179	20853	1071
1180	20853	1072
1181	20853	1073
1182	20853	1074
1183	20853	1075
1184	20853	1076
1185	20853	1077
1186	20853	1078
1187	20853	1079
1188	20853	1080
1189	20853	1081
1190	20853	1082
1191	20853	1083
1192	20853	1084
1193	20853	1085
1194	20853	1086
1195	20853	1087
1196	20853	1088
1197	20853	1089
1198	20853	1090
1199	20853	1091
1200	20853	1092
1201	20853	1093
1202	20853	1094
1203	20853	1095
1204	20853	1096
1205	20853	1097
1206	20853	1098
1207	20853	1099
1208	20853	1100
1209	20853	1101
1210	20853	1102
1211	20853	1103
1212	20853	1104
1213	20853	1105
1214	20853	1106
1215	20853	1107
1216	20853	1108
1217	20853	1109
1218	20853	1110
1219	20853	1111
1220	20853	1112
1221	20853	1113
1222	20853	1114
1223	20853	1115
1224	20853	1116
1225	20853	1117
1226	20853	1118
1227	20853	1119
1228	20853	1120
1229	20853	1121
1230	20853	1122
1231	20853	1123
1232	20853	1124
1233	208

1703	642653	42
1704	642653	43
1705	642653	44
1706	642653	45
1707	642653	46
1708	642653	47
1709	642653	48
1710	642653	49
1711	642653	50
1712	642653	51
1713	642653	52
1714	642653	53
1715	642653	54
1716	642653	55
1717	642653	56
1718	642653	57
1719	642653	58
1720	642653	59
1721	642653	60
1722	642653	61
1723	642653	62
1724	642653	63
1725	642653	64
1726	642653	65
1727	642653	66
1728	642653	67
1729	642653	68
1730	642653	69
1731	642653	70
1732	642653	71
1733	642653	72
1734	642653	73
1735	642653	74
1736	642653	75
1737	642653	76
1738	642653	77
1739	642653	78
1740	642653	79
1741	642653	80
1742	642653	81
1743	642653	82
1744	642653	83
1745	642653	84
1746	642653	85
1747	642653	86
1748	642653	87
1749	642653	88
1750	642653	89
1751	642653	90
1752	642653	91
1753	642653	92
1754	642653	93
1755	642653	94
1756	642653	95
1757	642653	96
1758	642653	97
1759	642653	98
1760	642653	99
1761	642653	100
1762	642653	101
1763	642653	102
1764	642653	103
1765	642653	104
1766	642653	105
1767	642653	106
1768	642653	107
17

2236	602287	279
2237	602287	280
2238	602287	281
2239	602287	282
2240	602287	283
2241	602287	284
2242	602287	285
2243	602287	286
2244	602287	287
2245	602287	288
2246	602287	289
2247	602287	290
2248	602287	291
2249	602287	292
2250	602287	293
2251	602287	294
2252	602287	295
2253	602287	296
2254	698918	1
2255	698918	2
2256	698918	3
2257	698918	4
2258	698918	5
2259	698918	6
2260	698918	7
2261	698918	8
2262	698918	9
2263	698918	10
2264	698918	11
2265	698918	12
2266	698918	13
2267	698918	14
2268	698918	15
2269	698918	16
2270	698918	17
2271	698918	18
2272	698918	19
2273	698918	20
2274	698918	21
2275	698918	22
2276	698918	23
2277	698918	24
2278	698918	25
2279	698918	26
2280	698918	27
2281	698918	28
2282	698918	29
2283	698918	30
2284	698918	31
2285	698918	32
2286	698918	33
2287	698918	34
2288	698918	35
2289	698918	36
2290	698918	37
2291	698918	38
2292	698918	39
2293	698918	40
2294	698918	41
2295	698918	42
2296	698918	43
2297	698918	44
2298	698918	45
2299	698918	46
2300	698918	47
2301	698918	48
2

2770	654138	195
2771	654138	196
2772	654138	197
2773	654138	198
2774	654138	199
2775	654138	200
2776	654138	201
2777	654138	202
2778	654138	203
2779	654138	204
2780	654138	205
2781	654138	206
2782	654138	207
2783	654138	208
2784	654138	209
2785	654138	210
2786	654138	211
2787	654138	212
2788	654138	213
2789	654138	214
2790	654138	215
2791	654138	216
2792	738174	1
2793	738174	2
2794	738174	3
2795	738174	4
2796	738174	5
2797	738174	6
2798	738174	7
2799	738174	8
2800	738174	9
2801	738174	10
2802	738174	11
2803	738174	12
2804	738174	13
2805	738174	14
2806	738174	15
2807	738174	16
2808	738174	17
2809	738174	18
2810	738174	19
2811	738174	20
2812	741449	1
2813	741449	2
2814	741449	3
2815	741449	4
2816	741449	5
2817	741449	6
2818	741449	7
2819	741449	8
2820	741449	9
2821	741449	10
2822	400739	1
2823	400739	2
2824	400739	3
2825	400739	4
2826	400739	5
2827	400739	6
2828	400739	7
2829	400739	8
2830	400739	9
2831	400739	10
2832	400739	11
2833	400739	12
2834	400739	13
2835	400739	14
2836	400739	15


3299	676695	102
3300	676695	103
3301	676695	104
3302	676695	105
3303	676695	106
3304	676695	107
3305	676695	108
3306	676695	109
3307	676695	110
3308	676695	111
3309	676695	112
3310	676695	113
3311	676695	114
3312	676695	115
3313	676695	116
3314	676695	117
3315	676695	118
3316	676695	119
3317	676695	120
3318	676695	121
3319	676695	122
3320	676695	123
3321	676695	124
3322	676695	125
3323	676695	126
3324	676695	127
3325	676695	128
3326	676695	129
3327	676695	130
3328	676695	131
3329	676695	132
3330	676695	133
3331	676695	134
3332	676695	135
3333	676695	136
3334	676695	137
3335	676695	138
3336	676695	139
3337	676695	140
3338	676695	141
3339	676695	142
3340	676695	143
3341	676695	144
3342	676695	145
3343	676695	146
3344	676695	147
3345	676695	148
3346	676695	149
3347	676695	150
3348	676695	151
3349	676695	152
3350	676695	153
3351	676695	154
3352	676695	155
3353	676695	156
3354	676695	157
3355	676695	158
3356	676695	159
3357	676695	160
3358	676695	161
3359	676695	162
3360	676695	163
3361	676

3837	723046	59
3838	723046	60
3839	723046	61
3840	723046	62
3841	723046	63
3842	723046	64
3843	723046	65
3844	723046	66
3845	723046	67
3846	710758	1
3847	710758	2
3848	710758	3
3849	710758	4
3850	710758	5
3851	710758	6
3852	710758	7
3853	710758	8
3854	710758	9
3855	710758	10
3856	710758	11
3857	710758	12
3858	710758	13
3859	710758	14
3860	710758	15
3861	710758	16
3862	710758	17
3863	710758	18
3864	710758	19
3865	710758	20
3866	710758	21
3867	710758	22
3868	710758	23
3869	710758	24
3870	710758	25
3871	710758	26
3872	710758	27
3873	710758	28
3874	710758	29
3875	710758	30
3876	710758	31
3877	710758	32
3878	710758	33
3879	710758	34
3880	710758	35
3881	710758	36
3882	710758	37
3883	710758	38
3884	710758	39
3885	710758	40
3886	710758	41
3887	710758	42
3888	710758	43
3889	710758	44
3890	710758	45
3891	710758	46
3892	710758	47
3893	710758	48
3894	710758	49
3895	710758	50
3896	710758	51
3897	710758	52
3898	710758	53
3899	710758	54
3900	710758	55
3901	710758	56
3902	710758	57
3903	710758	58
3904

4391	720810	16
4392	720810	17
4393	720810	18
4394	720810	19
4395	720810	20
4396	720810	21
4397	720810	22
4398	720810	23
4399	720810	24
4400	720810	25
4401	720810	26
4402	720810	27
4403	720810	28
4404	720810	29
4405	720810	30
4406	720810	31
4407	720810	32
4408	720810	33
4409	720810	34
4410	720810	35
4411	720810	36
4412	720810	37
4413	720810	38
4414	720810	39
4415	720810	40
4416	720810	41
4417	720810	42
4418	720810	43
4419	720810	44
4420	720810	45
4421	720810	46
4422	720810	47
4423	720810	48
4424	720810	49
4425	720810	50
4426	720810	51
4427	720810	52
4428	720810	53
4429	720810	54
4430	720810	55
4431	720810	56
4432	720810	57
4433	720810	58
4434	720810	59
4435	720810	60
4436	720810	61
4437	720810	62
4438	720810	63
4439	720810	64
4440	720810	65
4441	720810	66
4442	720810	67
4443	720810	68
4444	720810	69
4445	720810	70
4446	720810	71
4447	720810	72
4448	720810	73
4449	738144	1
4450	738144	2
4451	738144	3
4452	738144	4
4453	738144	5
4454	738144	6
4455	738144	7
4456	738144	8
4457	738144	9
4458

## 18세 이상 이용 웹툰 댓글 크롤링
- 셀레니움으로 로그인시, 캡차를 무력화 하기위해 여러가지 시도를 해봤지만 모두 실패ㅜㅜ
- 로그인은 수동으로 해야함...

In [26]:
df = pd.read_csv('webtoon_link.csv')
len(df)

325

In [10]:
df = df[df.age == 18]
df

,link1,title,weekday,titleId,last,content,genre,age
32,https://comic.naver.com/webtoon/list.nhn?title...,닥터 하운드,mon,700858,139,인간의 생각과 감정 그리고 죽음의 흔적까지꿰뚫어 보는 눈을 가진 남자 '하운드'그 ...,"스토리, 스릴러",18
73,https://comic.naver.com/webtoon/list.nhn?title...,헬58,tue,719508,76,"약육강식의 섬 헬58에서 벌어지는 권력다툼과 암투, 배신. 주인공 서민은 이 지옥같...","스토리, 액션",18
77,https://comic.naver.com/webtoon/list.nhn?title...,피와 살,tue,730148,44,멸종 위기에 처한 인간.인간을 먹고 사는 괴물.생존을 위한 그들의 전쟁 혹은 공생.,"스토리, 스릴러",18
95,https://comic.naver.com/webtoon/list.nhn?title...,헬퍼 2 : 킬베로스,wed,670143,223,역시는 역시 역시군.킬베로스 트라이브의 대장 장광남이 펼치는 스트릿 액션 만화.,"스토리, 액션",18
219,https://comic.naver.com/webtoon/list.nhn?title...,사냥개들,fri,727784,52,"빚 때문에 얽힌 세 명의 젊은이들, 그들이 보여주는 더러운 돈 사냥이 시작된다.","스토리, 액션",18
262,https://comic.naver.com/webtoon/list.nhn?title...,토끼대왕,sat,742826,7,"살인마 교생, 왕따를 구원하다.","스토리, 스릴러",18
264,https://comic.naver.com/webtoon/list.nhn?title...,압락사스,sat,729563,39,"""다 죽여버리면 되는 거잖아""꼬일 대로 꼬인 악의 연대기","스토리, 스릴러",18
287,https://comic.naver.com/webtoon/list.nhn?title...,스퍼맨: 현자단의 역습,sun,730259,44,스퍼맨이 돌아왔다! 근데 현자가 되어 돌아왔다!,"에피소드, 드라마",18


In [22]:
link1 = list (df.link1)
info = []
for link in link1:
    response = requests.get(link)
    dom = BeautifulSoup(response.content, "html.parser")
    last = dom.select('.title > a')[0].get('href')
    age = dom.select('div.detail > p.detail_info > span.age')[0].text[:2]
    info.append({'titleId': int(re.findall('\d+', last)[0]),
             'last': int(re.findall('\d+', last)[1]),
            })

In [24]:
df = pd.merge(df, pd.DataFrame(info))
df

,link1,title,weekday,titleId,last,content,genre,age
0,https://comic.naver.com/webtoon/list.nhn?title...,닥터 하운드,mon,700858,139,인간의 생각과 감정 그리고 죽음의 흔적까지꿰뚫어 보는 눈을 가진 남자 '하운드'그 ...,"스토리, 스릴러",18
1,https://comic.naver.com/webtoon/list.nhn?title...,헬58,tue,719508,76,"약육강식의 섬 헬58에서 벌어지는 권력다툼과 암투, 배신. 주인공 서민은 이 지옥같...","스토리, 액션",18
2,https://comic.naver.com/webtoon/list.nhn?title...,피와 살,tue,730148,44,멸종 위기에 처한 인간.인간을 먹고 사는 괴물.생존을 위한 그들의 전쟁 혹은 공생.,"스토리, 스릴러",18
3,https://comic.naver.com/webtoon/list.nhn?title...,헬퍼 2 : 킬베로스,wed,670143,223,역시는 역시 역시군.킬베로스 트라이브의 대장 장광남이 펼치는 스트릿 액션 만화.,"스토리, 액션",18
4,https://comic.naver.com/webtoon/list.nhn?title...,사냥개들,fri,727784,52,"빚 때문에 얽힌 세 명의 젊은이들, 그들이 보여주는 더러운 돈 사냥이 시작된다.","스토리, 액션",18
5,https://comic.naver.com/webtoon/list.nhn?title...,토끼대왕,sat,742826,7,"살인마 교생, 왕따를 구원하다.","스토리, 스릴러",18
6,https://comic.naver.com/webtoon/list.nhn?title...,압락사스,sat,729563,39,"""다 죽여버리면 되는 거잖아""꼬일 대로 꼬인 악의 연대기","스토리, 스릴러",18
7,https://comic.naver.com/webtoon/list.nhn?title...,스퍼맨: 현자단의 역습,sun,730259,44,스퍼맨이 돌아왔다! 근데 현자가 되어 돌아왔다!,"에피소드, 드라마",18


In [25]:
last_ls = []
for i in range(len(df)):
    last_ls.append(list(df[['titleId', 'last', 'weekday']].iloc[i]))

# link2 : 각 웹툰의 모든 회차 링크
link2 = []
for last in last_ls:
    for no in range(1, int(last[1])+1):
        link = f'https://comic.naver.com/webtoon/detail.nhn?titleId={last[0]}&no={no}&weekday={last[2]}'
        link2.append(link) 

In [41]:
len(link2)

624

#### 로그인

In [42]:
driver = webdriver.Chrome()
driver.get('https://nid.naver.com/nidlogin.login')

- 이렇게 하면 좋지만,,, 실패했으므로 수동로그인 해야한다


`!cat config.json`

```
{
    "ID": "naver_id",
    "PW": "naver_pw"
}
```
---
```python
def get_config():
    try:
        with open('config.json') as json_file:
            json_data = json.load(json_file)
    except Exception as e:
        print(f'Error in reading config file, {e}')
        return None
    else:
        return json_data
    
config = get_config()
id_ = config['ID']
pw_ = config['PW']

driver.find_element_by_css_selector('#id').send_keys(id_)
time.sleep(3)
driver.find_element_by_css_selector('#pw').send_keys(pw)
time.sleep(3)
driver.find_element_by_css_selector('#log\.login').click()
```

#### 댓글 크롤링

In [43]:
def get_comments(links):
    comments = []
    i = 1
    for link in links:
        driver.get(link)
        time.sleep(0.5)
        driver.switch_to_frame('commentIframe')
        page_source = driver.page_source

        dom = BeautifulSoup(page_source, 'html.parser')
        coms = dom.select('.u_cbox_contents')

        reply = []
        for com in coms:
            reply.append(com.text)
        titleId = re.findall('\d+', link)[0]
        no = re.findall('\d+', link)[1]
        comments.append({'titleId': titleId,
                            'no': no,
                            'comments': ', '.join(reply)})
        print(i, titleId, no, sep='\t')
        i += 1
    return comments

In [44]:
start = time.time()
comments = get_comments(link2)
pd.DataFrame(comments).to_csv('./comments/naver_over18.csv', index=False, encoding='utf-8')
print(f'time: {time.time() - start}')

1	700858	1
2	700858	2
3	700858	3
4	700858	4
5	700858	5
6	700858	6
7	700858	7
8	700858	8
9	700858	9
10	700858	10
11	700858	11
12	700858	12
13	700858	13
14	700858	14
15	700858	15
16	700858	16
17	700858	17
18	700858	18
19	700858	19
20	700858	20
21	700858	21
22	700858	22
23	700858	23
24	700858	24
25	700858	25
26	700858	26
27	700858	27
28	700858	28
29	700858	29
30	700858	30
31	700858	31
32	700858	32
33	700858	33
34	700858	34
35	700858	35
36	700858	36
37	700858	37
38	700858	38
39	700858	39
40	700858	40
41	700858	41
42	700858	42
43	700858	43
44	700858	44
45	700858	45
46	700858	46
47	700858	47
48	700858	48
49	700858	49
50	700858	50
51	700858	51
52	700858	52
53	700858	53
54	700858	54
55	700858	55
56	700858	56
57	700858	57
58	700858	58
59	700858	59
60	700858	60
61	700858	61
62	700858	62
63	700858	63
64	700858	64
65	700858	65
66	700858	66
67	700858	67
68	700858	68
69	700858	69
70	700858	70
71	700858	71
72	700858	72
73	700858	73
74	700858	74
75	700858	75
76	700858	76
77	700858	77
78	700858	78
79	7

587	730259	7
588	730259	8
589	730259	9
590	730259	10
591	730259	11
592	730259	12
593	730259	13
594	730259	14
595	730259	15
596	730259	16
597	730259	17
598	730259	18
599	730259	19
600	730259	20
601	730259	21
602	730259	22
603	730259	23
604	730259	24
605	730259	25
606	730259	26
607	730259	27
608	730259	28
609	730259	29
610	730259	30
611	730259	31
612	730259	32
613	730259	33
614	730259	34
615	730259	35
616	730259	36
617	730259	37
618	730259	38
619	730259	39
620	730259	40
621	730259	41
622	730259	42
623	730259	43
624	730259	44
time: 1283.6282618045807


In [ ]:
driver.quit()

## 데이터 불러오기

In [35]:
webtoon_df = pd.read_csv('./comments/webtoon_link.csv')
webtoon_df.tail(1)

,link1,title,weekday,titleId,last,content,genre,age
324,https://comic.naver.com/webtoon/list.nhn?title...,호곡,sun,713055,97,'이제 우리 차례야'조선의 왕 '천군'이 사라지고 3년 뒤...한 소녀가 나타나다.,"스토리, 판타지",12


In [33]:
mon = webtoon_df[(webtoon_df.weekday == 'mon')&(webtoon_df.age < 18)]
tue = webtoon_df[(webtoon_df.weekday == 'tue')&(webtoon_df.age < 18)]
wed = webtoon_df[(webtoon_df.weekday == 'wed')&(webtoon_df.age < 18)]
thu = webtoon_df[(webtoon_df.weekday == 'thu')&(webtoon_df.age < 18)]
fri = webtoon_df[(webtoon_df.weekday == 'fri')&(webtoon_df.age < 18)]
sat = webtoon_df[(webtoon_df.weekday == 'sat')&(webtoon_df.age < 18)]
sun = webtoon_df[(webtoon_df.weekday == 'sun')&(webtoon_df.age < 18)]
over18 = webtoon_df[webtoon_df.age == 18]
len(mon), len(tue), len(wed), len(thu), len(fri), len(sat), len(sun), len(over18)

(46, 43, 43, 46, 46, 44, 49, 8)

In [37]:
comment_mon = pd.read_csv('./comments/naver_under18_mon.csv')
comment_tue = pd.read_csv('./comments/naver_under18_tue.csv')
comment_wed = pd.read_csv('./comments/naver_under18_wed.csv')
comment_thu = pd.read_csv('./comments/naver_under18_thu.csv')
comment_fri = pd.read_csv('./comments/naver_under18_fri.csv')
comment_sat = pd.read_csv('./comments/naver_under18_sat.csv')
comment_sun = pd.read_csv('./comments/naver_under18_sun.csv')
comment_18 = pd.read_csv('./comments/naver_over18.csv')
comment_18.tail(1)

,titleId,no,comments
623,730259,44,"어려운 포즈를 도와주신 액션피겨 분들 에게 무한한 감사를 드립니다, 액션피겨? 저분..."


In [30]:
len(comment_mon), len(comment_tue), len(comment_wed), len(comment_thu), len(comment_fri), len(comment_sat), len(comment_sun), len(comment_18)

(4025, 4603, 3345, 3727, 4061, 4045, 4403, 624)

In [62]:
columns = ['title', 'weekday', 'last', 'content', 'genre', 'age', 'no', 'comments']

In [61]:
df_mon = pd.merge(mon, comment_mon, on='titleId', how='outer')[columns]
df_tue = pd.merge(tue, comment_tue, on='titleId', how='outer')[columns]
df_wed = pd.merge(wed, comment_wed, on='titleId', how='outer')[columns]
df_thu = pd.merge(thu, comment_thu, on='titleId', how='outer')[columns]
df_fri = pd.merge(fri, comment_fri, on='titleId', how='outer')[columns]
df_sat = pd.merge(sat, comment_sat, on='titleId', how='outer')[columns]
df_sun = pd.merge(sun, comment_sun, on='titleId', how='outer')[columns]
df_18 = pd.merge(over18, comment_18, on='titleId', how='outer')[columns]

In [60]:
naver_comment = pd.concat([df_mon, df_tue, df_wed, df_thu, df_fri, df_sat, df_sun, df_18]).reset_index(drop=True)
naver_comment.tail()

,title,weekday,last,content,genre,age,no,comments
28828,스퍼맨: 현자단의 역습,sun,44,스퍼맨이 돌아왔다! 근데 현자가 되어 돌아왔다!,"에피소드, 드라마",18,40,"애무투에 이은 페니싱ㅋㅋㅋㅋ, 발기토 미치겠네, 이것이 바로 그 전설의좃.검.승.부..."
28829,스퍼맨: 현자단의 역습,sun,44,스퍼맨이 돌아왔다! 근데 현자가 되어 돌아왔다!,"에피소드, 드라마",18,41,"ㅁㅊ 하일권 뇌엔 뭐가 들었길래 이런 아이디어가 나오는거지, 근데 진짜 애무투때부터..."
28830,스퍼맨: 현자단의 역습,sun,44,스퍼맨이 돌아왔다! 근데 현자가 되어 돌아왔다!,"에피소드, 드라마",18,42,"연산 : 헤어져요 우리, 작아서 싫어요, 결과만보면 축소수술 30년동안 한거네 ....."
28831,스퍼맨: 현자단의 역습,sun,44,스퍼맨이 돌아왔다! 근데 현자가 되어 돌아왔다!,"에피소드, 드라마",18,43,"예언자님도 행복하게해주세요ㅠㅠ...그렇게 저희의눈(?)을 즐겁게 해주셨는데....,..."
28832,스퍼맨: 현자단의 역습,sun,44,스퍼맨이 돌아왔다! 근데 현자가 되어 돌아왔다!,"에피소드, 드라마",18,44,"어려운 포즈를 도와주신 액션피겨 분들 에게 무한한 감사를 드립니다, 액션피겨? 저분..."


In [63]:
naver_comment.to_csv('naver_best_comment.csv', index=False, encoding='utf-8')